In [13]:
import time
import numpy as np
from Conways import*

In [14]:
env = conways(size = 10, max_it=1000)
print(env.run())

60


In [15]:
x = np.zeros(shape=(10, 10), dtype=bool)
x[1, 1] = True
x[1, 2] = True
x[2, 1] = True
x[2, 2] = True
env.reset(x, offset=False)
actions = env.run()
print(actions)

inf


In [24]:
alpha = 0.1 
max_iterations = 6000
gamma = 0.995

total = 0
tot_reward = 0
prev_reward = 0

q_table_row_limit = 75

epsilon = 1
epsilon_decay = 0.99995

In [25]:
q_table = np.random.uniform(low=0, high=1, size=[q_table_row_limit, 10, 10])
q_table.shape

print(np.argmax(q_table[0]))
print(np.max(q_table[0]))
print(q_table[0])

6
0.9842075341843078
[[0.95492959 0.37671253 0.04536318 0.57877887 0.66438921 0.62724743
  0.98420753 0.19849429 0.32189768 0.17142772]
 [0.93433593 0.6175325  0.58618332 0.89215819 0.88559545 0.03814588
  0.93467854 0.04722136 0.42865486 0.44703616]
 [0.714715   0.66056588 0.0942604  0.04037582 0.96779647 0.32861197
  0.92515708 0.8765821  0.75992617 0.05860798]
 [0.33936413 0.07128884 0.68027522 0.81830081 0.22743049 0.5757405
  0.70008274 0.70661668 0.0625239  0.16593046]
 [0.75723691 0.05798787 0.16292386 0.54656086 0.34798516 0.04638192
  0.25812251 0.56492204 0.90276983 0.01877305]
 [0.68686922 0.07475778 0.63495469 0.12811669 0.80516797 0.02123285
  0.27523412 0.7226701  0.00853195 0.44656618]
 [0.76768596 0.8449811  0.8994773  0.88686095 0.86954444 0.19896496
  0.52805697 0.63392875 0.23910031 0.88197276]
 [0.48505539 0.7711462  0.83897698 0.09170749 0.2672924  0.85002207
  0.83418384 0.03880016 0.88865216 0.63834232]
 [0.12008328 0.21301726 0.22381916 0.19035045 0.62682385 0.7

In [26]:

state_score = np.array([float('-inf') for i in range(q_table_row_limit)])
state_table = np.random.uniform(low=0, high=1, size=[q_table_row_limit, 10, 10]) 

def get_discrete_state(s):
    for i in range(q_table_row_limit):
        if np.array_equal(s, state_table[i]):
            return i
    i = np.argmin(state_score)
    state_table[i] = np.copy(s)
    q_table[i] = np.random.uniform(low=0, high=1, size=[10, 10])
    return i


In [27]:
current_state = np.zeros(shape=(10, 10), dtype=bool)

#take initial action
discrete_state = get_discrete_state(current_state)

action = np.argmax(q_table[discrete_state])
action = (int(action/10), action%10)
current_state[action] = True
best_state = current_state
env.reset(current_state, offset = False)
state_score[discrete_state] = env.run()
best_reward = float('-inf')

for i in range(max_iterations):
    t0 = time.time()
    

    if (i % 2000 == 0):
        print(f"Episode: {i}")

    reward = 0
    #test possible actions after taking that action
    for j in range(q_table_row_limit):
        if np.random.random() > epsilon:
            action = np.argmax(q_table[discrete_state])
        else:
            action = np.random.randint(0, 100)
        action = (int(action/10), action%10)
        current_state[action] = True
        env.reset(current_state, offset = False)

        new_discrete_state = get_discrete_state(current_state)

        state_score[discrete_state] = env.run()
        reward += state_score[discrete_state]

        if (state_score[discrete_state] > best_reward):
            best_state = np.copy(current_state)
            best_reward = state_score[discrete_state]

        max_future_q = np.max(q_table[new_discrete_state])
        current_q = q_table[(discrete_state, action)]

        new_q = (1 - alpha) * current_q + alpha * (state_score[discrete_state] + gamma * max_future_q)
        q_table[(discrete_state, action)] = new_q

        discrete_state = new_discrete_state
    
    
    current_state = np.copy(best_state)
    reward = np.max(state_score)
    if epsilon > 0.05: 
        if reward > prev_reward and i > 10000:
            epsilon = epsilon_decay ** (i - 10000)

            if i % 500 == 0:
                print("Epsilon: " + str(epsilon))
    
    t1 = time.time()
    iteration_time = t1-t0
    total += iteration_time

    tot_reward += reward
    prev_reward = reward

    if i % 1000 == 0: 
        mean = total / 1000
        print("Time Average: " + str(mean))
        total = 0

        mean_reward = tot_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        tot_reward = 0

Episode: 0
Time Average: 0.0002559924125671387
Mean Reward: 0.069
Time Average: 0.2759894394874573
Mean Reward: 134.492
Episode: 2000
Time Average: 0.2739965097904205
Mean Reward: 136.0
Time Average: 0.2656098198890686
Mean Reward: 136.0
Episode: 4000
Time Average: 0.2708282732963562
Mean Reward: 136.0
Time Average: 0.27518734884262086
Mean Reward: 136.0
Episode: 6000
Time Average: 0.2622783043384552
Mean Reward: 136.0
Time Average: 0.2592230875492096
Mean Reward: 136.0
Episode: 8000
Time Average: 0.2626591453552246
Mean Reward: 136.0
Time Average: 0.25234042716026306
Mean Reward: 136.0


In [29]:
Final = best_state
print(Final)
env.reset(Final, offset = False)
env.render = True
print(env.run())

[[ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True False  True  True  True  True  True  True  True  True]
 [False  True False False False False  True  True  True  True]
 [ True False False False False  True  True  True  True  True]
 [False False  True  True  True False  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]]
136
